In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from utils.get_summary_data import compile_summary
from utils.paths import SetupPaths

In [2]:
plt.rcParams.update({'font.size':22})
paths = SetupPaths()


In [3]:
style = {"dark":"dashed","hydro":"solid"}
palette = { "dark":"#044751","hydro": "#B6DAFF", 
           "Illustris":"#668AFF","TNG": "#B6DAFF", # "TNG": "#7B55F3"
           "Illustris dark": "#009292", "Illustris hydro": "#B6DAFF",
           "TNG dark": "#930200", "TNG hydro": "#FFB5DC",
           "All Pairs":"#668AFF","Major Pairs": "#7B55F3",
           "Minor Pairs":'#FFB201',"Minor Plus Pairs":"#FF6300",
           "Extra":"#DE2280","Stellar":"#0A68DD",
           "Halo":"#4B0094","Max Halo":"#FF6BB7"}
alphaVal = {"Major":0.7,"Minor":0.4}

Goal:

We have a collection of halo pairs in hydro, in dark, and we'd like to pull out the subsample of hydro that have matches in dark. 

Want: major and minor pairs, primary data, dwarf, massive, etc.

In [4]:
# make functions to get data at the requested snapshot
def get_primmask(primstells, size):
    if size == "dwarf":
        mask = (primstells > 0.01) & (primstells < 0.5)
    elif size == "massive":
        mask = (primstells > 0.5) & (primstells < 10)
    return mask

def get_groupmask(groupmass, size):
    if size == "dwarf":
        mask = (groupmass > 8) & (groupmass < 50)
    elif size == "massive":
        mask = (groupmass > 100) & (groupmass < 650)
    return mask

# Start by getting ids for TNG hydro snap 99 dwarf primaries

In [5]:
sim = "TNG"
snap = 99
phys = "hydro"
size = "dwarf"

In [6]:
pair_path = f"{sim}_{snap}_100.hdf5"
pair_data = h5py.File(f"{paths.path_pairs}{pair_path}", "r")


make catalog of all unpaired primaries:

In [7]:
pair_data['unpaired'][phys].keys()

<KeysViewHDF5 ['Group ID', 'Group Mass', 'Group Nsubs', 'Group Radius', 'Realization', 'Sub1 BHMass', 'Sub1 BHMdot', 'Sub1 GasMetallicity', 'Sub1 ID', 'Sub1 Mass', 'Sub1 MassType', 'Sub1 Pos', 'Sub1 SFR', 'Sub1 SFRinRad', 'Sub1 Stellar Mass', 'Sub1 Vel']>

In [8]:
subids = np.array(pair_data['unpaired'][phys]['Sub1 ID'])
grmass = np.array(pair_data['unpaired'][phys]['Group Mass'])
stmass = np.array(pair_data['unpaired'][phys]['Sub1 Stellar Mass'])


In [9]:
grmask = get_groupmask(grmass, size)        
stmask = get_primmask(stmass, size)

totmask = stmask & grmask

In [10]:
ids_pass = subids[totmask]

In [11]:
ids_pass

array([569290, 569827, 571131, ..., 718824, 718824, 718824])

In [12]:
matchedN = h5py.File(f"../../IllustrisTNG/TNG100-Matched-Nelson/subhalo_matching_to_dark.hdf5", "r")
darkidsN = np.array(matchedN['Snapshot_99']['SubhaloIndexDark_SubLink'])
dark_ids_passN = darkidsN[ids_pass]

In [14]:
matchedV = h5py.File(f"../../IllustrisTNG/TNG100-Matched-V/dm_match_099.hdf5", "r")
matchedV.keys()

<KeysViewHDF5 ['IndexFrom', 'IndexTo']>

"IndexTo" is the DMO subhalo ID

"IndexFrom" is the hydro ID

In [57]:
darkidsV = np.array(matchedV["IndexTo"])
hydroidsV = np.array(matchedV["IndexFrom"])

# Vicente catalog will skip indices of halos, so IndexFrom will give you the hydro ID, but the hydroID != the index. 
# I have a list of indices, so I need to find the corresponding index in hydroidsV:
hydroidsmask = hydroidsV.searchsorted(ids_pass)

dark_ids_passV = darkidsV[hydroidsmask]

Remember that I should save the fraction of matched halos (len(ids_pass)-np.sum( dark_ids_pass == -1 ))/(len(ids_pass))

In [69]:
dark_ids_passN[0:10]

array([817812, 809420, 808348, 811542, 811542, 803611, 800305, 805889,
       805889, 807630], dtype=int32)

In [70]:
dark_ids_passV[0:10]

array([817812, 809420, 808348, 811542, 811542, 803611, 800305, 805889,
       805889, 807630], dtype=int32)

In [63]:
sum(dark_ids_passV!=dark_ids_passN)

101

___

___

In [ ]:
 
    redshifts, numAll, numPri, numMaj, numMin = [], [], [], [], []
    fracAll, fracMaj, fracMin = [], [], []
    quartAll, quartPri, quartMaj, quartMin = [], [], [], []
    quartfracAll, quartfracMaj, quartfracMin = [], [], []
    
    sepcut = kwargs.pop('sepcut',None)
       
    for snap in snapshots:  
        try:
            
            
            if np.size(pair_data) == 0:
                raise EmptyFile
                
            redshift = pair_data['Header'].attrs['Redshift']
            
            if redshiftcutoff & ( redshift > redcutoff) :
                raise SkipRedshift
                
            if (len(pair_data['pairs'][phys]['Group ID']) == 0):    
                raise EmptyFile
                
            unpair = pair_data["unpaired"][phys]
            unpairStells = np.array(unpair["Sub1 Stellar Mass"])
            unpairGroups = np.array(unpair["Group Mass"])
            unpairReals = np.array(unpair['Realization'])
            
            pair = pair_data["pairs"][phys]
            priStell = np.array(pair["Sub1 Stellar Mass"])
            secStell = np.array(pair["Sub2 Stellar Mass"])
            pairGroups = np.array(pair["Group Mass"])
            pairReals = np.array(pair["Realization"])
            seps = np.array(pair["Separation"]) 
            
            # subset masks for unpaired
            unpair_pri = get_primmask(unpairStells, size)
            unpair_group = get_groupmask(unpairGroups, size)                
            
            pair_pri = get_primmask(priStell, size)
            pair_group = get_groupmask(pairGroups, size)

            majors = (secStell/priStell > 1/4)
            minors = (secStell/priStell > 1/10) & (secStell/priStell < 1/4)
            pair_lowsep = (seps > 10) & (seps < 10000)

            # defining combined masks 
            unpair_mask = unpair_pri & unpair_group
            pair_mask = pair_pri & pair_group & pair_lowsep
                
            if sepcut == 'lower':
                lower = kwargs.pop("lowerthan",100)
                pair_lower = seps < lower
                pair_mask = pair_mask & pair_lower
            elif sepcut == 'upper':
                upper = kwargs.pop("higherthan",100)
                pair_higher = seps > upper
                pair_mask = pair_mask & pair_higher
                               
            pri_perreal, all_perreal, maj_perreal, min_perreal = [], [], [], []
            fracall_perreal, fracmaj_perreal, fracmin_perreal = [], [], []
                
            realizations = np.unique( unpairReals )
                
            for real in realizations:                  
                # make realization masks
                unpair_real = unpairReals == real
                pair_real = pairReals == real

                # make copies of masks
                unpair_mask_real = unpair_mask & unpair_real
                pair_mask_real = pair_mask & pair_real

                majormask_real = pair_mask_real & majors & pair_lowsep
                minormask_real = pair_mask_real & minors & pair_lowsep

                num_unpair_real = np.count_nonzero(unpair_mask_real)
                num_pair_real = np.count_nonzero(pair_mask_real)   
                num_maj_real = np.count_nonzero(majormask_real)   
                num_min_real = np.count_nonzero(minormask_real)  
                num_pri_real = num_unpair_real + num_pair_real

                pri_perreal.append( num_pri_real )
                all_perreal.append( num_pair_real )
                maj_perreal.append( num_maj_real )
                min_perreal.append( num_min_real )
                fracall_perreal.append( num_pair_real / num_pri_real )
                fracmaj_perreal.append( num_maj_real / num_pri_real )
                fracmin_perreal.append( num_min_real / num_pri_real )

            numPri_z = np.median( pri_perreal )
            numAll_z = np.median( all_perreal )
            numMaj_z = np.median( maj_perreal )
            numMin_z = np.median( min_perreal )
            fracAll_z = np.median( fracall_perreal )
            fracMaj_z = np.median( fracmaj_perreal )
            fracMin_z = np.median( fracmin_perreal )

            lower, upper = 1, 99

            quartPri_z = np.percentile( pri_perreal, [lower,upper])
            quartAll_z = np.percentile( all_perreal, [lower,upper])
            quartMaj_z = np.percentile( maj_perreal, [lower,upper])
            quartMin_z = np.percentile( min_perreal, [lower,upper])
            quartfracAll_z = np.percentile( fracall_perreal, [lower,upper])
            quartfracMaj_z = np.percentile( fracmaj_perreal, [lower,upper])
            quartfracMin_z = np.percentile( fracmin_perreal, [lower,upper])
            
            redshifts.append( redshift )
            numPri.append( numPri_z )
            numAll.append( numAll_z )
            numMaj.append( numMaj_z )
            numMin.append( numMin_z )
            fracAll.append( fracAll_z )
            fracMaj.append( fracMaj_z )
            fracMin.append( fracMin_z )
            quartPri.append( quartPri_z )
            quartAll.append( quartAll_z )
            quartMaj.append( quartMaj_z )
            quartMin.append( quartMin_z )
            quartfracAll.append( quartfracAll_z )
            quartfracMaj.append( quartfracMaj_z )
            quartfracMin.append( quartfracMin_z )
                       
        except KeyError:
            if errorprint: print(f'skipping {snap} for KeyError. Please debug')
            continue
            
        except EmptyFile:
            if errorprint: print(f"skipping {snap}, empty file")
            continue
            
        except SkipRedshift:
            if errorprint: print(f"skipping {snap}, redshift out of range")
                
    count_dictionary = {
            "z": np.array(redshifts),
            "Number Primaries": np.array(numPri),
            "Number Pairs": np.array(numAll),
            "Number Majors": np.array(numMaj),
            "Number Minors": np.array(numMin),
            "Fraction Pairs": np.array(fracAll),
            "Fraction Majors": np.array(fracMaj),
            "Fraction Minors": np.array(fracMin),
            "Quarts Primaries": np.array(quartPri),
            "Quarts Pairs": np.array(quartAll),
            "Quarts Majors": np.array(quartMaj),
            "Quarts Minors": np.array(quartMin),
            "Quarts Fraction Pairs": np.array(quartfracAll),
            "Quarts Fraction Majors": np.array(quartfracMaj),
            "Quarts Fraction Minors": np.array(quartfracMin)}
            
    return count_dictionary

def get_paircounts_simstell(sim, size, errorprint=False, redshiftcutoff=True, **kwargs):    
    if sim == "Illustris":
        snapshots = np.arange(0,136,1)
        snapshots = np.delete(snapshots, np.where(snapshots==113)[0])
        redcutoff = 3.8
    elif sim == "TNG":
        snapshots = np.arange(0,100,1)
        snapshots = np.delete(snapshots, np.where(snapshots==48)[0])
        redcutoff = 4
        
    redshifts, numAll, numPri, numMaj, numMin = [], [], [], [], []
    fracAll, fracMaj, fracMin = [], [], []

    sepcut = kwargs.pop('sepcut',None)
       
    for snap in snapshots:  
        try:
            pair_path = f"{sim}_{snap}_simstars.hdf5"
            pair_data = h5py.File(f"{paths.path_pairs}{pair_path}", "r")
            
            if np.size(pair_data) == 0:
                raise EmptyFile
                
            redshift = pair_data['Header'].attrs['Redshift']
            
            if redshiftcutoff & ( redshift > redcutoff) :
                raise SkipRedshift
                
            if (len(pair_data['paired']["hydro"]['Group ID']) == 0):    
                raise EmptyFile
                
                
            unpair = pair_data["unpaired"]["hydro"]
            unpairStells = np.array(unpair["Sub1 Stellar Mass"])
            unpairGroups = np.array(unpair["Group Mass"])
            unpairReals = np.array(unpair['Realization'])
            
            pair = pair_data["paired"]["hydro"]
            priStell = np.array(pair["Sub1 Stellar Mass"])
            secStell = np.array(pair["Sub2 Stellar Mass"])
            pairGroups = np.array(pair["Group Mass"])
            pairGrRads = np.array(pair["Group Radius"])
            pairReals = np.array(pair["Realization"])
            seps = np.array(pair["Separation"])   
            
            # subset masks for unpaired
            unpair_pri = get_primmask(unpairStells, size)
            unpair_group = get_groupmask(unpairGroups, size)                
            unpair_medreal = unpairReals == -1
            
            pair_pri = get_primmask(priStell, size)
            pair_group = get_groupmask(pairGroups, size)
            pair_medreal = pairReals == -1

            majors = (secStell/priStell > 1/4)
            minors = (secStell/priStell > 1/10) & (secStell/priStell < 1/4)
            pair_lowsep = (seps > 10) & (seps < 10000)
        
            # defining combined masks 
            unpair_mask = unpair_pri & unpair_group
            pair_mask = pair_pri & pair_group & pair_lowsep 
            
            if sepcut == 'lower':
                lower = kwargs.pop('lowerthan',100)
                pair_lower = seps < lower
                pair_mask = pair_mask & pair_lower
            elif sepcut == 'upper':
                upper = kwargs.pop('higherthan',100)
                pair_higher = seps > upper
                pair_mask = pair_mask & pair_higher
                
            unpair_mask = unpair_mask & unpair_medreal
            pair_mask = pair_mask & pair_medreal
            
            major_mask = pair_mask & majors 
            minor_mask = pair_mask & minors
            
            num_unpair = np.count_nonzero(unpair_mask)
            num_pair = np.count_nonzero(pair_mask)   
            num_maj = np.count_nonzero(major_mask)   
            num_min = np.count_nonzero(minor_mask)   

            numPri_z = num_pair + num_unpair
            numAll_z = num_pair
            numMaj_z = num_maj
            numMin_z = num_min
            fracAll_z = numAll_z / numPri_z
            fracMaj_z = numMaj_z / numPri_z
            fracMin_z = numMin_z / numPri_z             
           
            redshifts.append( redshift )
            numPri.append( numPri_z )
            numAll.append( numAll_z )
            numMaj.append( numMaj_z )
            numMin.append( numMin_z )
            fracAll.append( fracAll_z )
            fracMaj.append( fracMaj_z )
            fracMin.append( fracMin_z )
                       
        
        except KeyError:
            if errorprint: print(f'skipping {snap} for KeyError. Please debug')
            continue
            
        except EmptyFile:
            if errorprint: print(f"skipping {snap}, empty file")
            continue
            
        except SkipRedshift:
            if errorprint: print(f"skipping {snap}, redshift out of range")
                
    count_dictionary = {
            "z": np.array(redshifts),
            "Number Primaries": np.array(numPri),
            "Number Pairs": np.array(numAll),
            "Number Majors": np.array(numMaj),
            "Number Minors": np.array(numMin),
            "Fraction Pairs": np.array(fracAll),
            "Fraction Majors": np.array(fracMaj),
            "Fraction Minors": np.array(fracMin)
    }
    
            
    return count_dictionary

def get_key(sim, phys, size, key, sepcut, errorprint=False, redshiftcutoff=True):    
    if sim == "Illustris":
        snapshots = np.arange(0,136,1)
        snapshots = np.delete(snapshots, np.where(snapshots==113)[0])
        redcutoff = 3.8
    elif sim == "TNG":
        snapshots = np.arange(0,100,1)
        snapshots = np.delete(snapshots, np.where(snapshots==48)[0])
        redcutoff = 4
        
    redshifts = []  
    med_pairs, med_major, med_minor = [], [], []
    quart_pairs, quart_major, quart_minor = [], [], []
            
    for snap in snapshots:  
        try:
            pair_path = f"{sim}_{snap}_10.hdf5"
            pair_data = h5py.File(f"{paths.path_pairs}{pair_path}", "r")
            
            if np.size(pair_data) == 0:
                raise EmptyFile
                
            redshift = pair_data['Header'].attrs['Redshift']
            
            if redshiftcutoff & ( redshift > redcutoff) :
                raise SkipRedshift
                
            if (len(pair_data['pairs'][phys]['Group ID']) == 0):    
                raise EmptyFile
                
            pair = pair_data["pairs"][phys]
            priStell = np.array(pair["Sub1 Stellar Mass"])
            secStell = np.array(pair["Sub2 Stellar Mass"])
            pairGroups = np.array(pair["Group Mass"])
            pairGrRads = np.array(pair["Group Radius"])
            pairReals = np.array(pair["Realization"])
            seps = np.array(pair["Separation"]) 
            keydat = np.array(pair[key])
            
            # subset masks for unpaired
            pair_pri = get_primmask(priStell, size)
            pair_group = get_groupmask(pairGroups, size)

            majors = (secStell/priStell > 1/4)
            minors = (secStell/priStell > 1/10) & (secStell/priStell < 1/4)
            pair_lowsep = (seps > 10) & (seps < 10000)

            # defining combined masks 
            pair_mask = pair_pri & pair_group & pair_lowsep
        
            if sepcut == "low":
                sep_mask = seps < 0.5*pairGrRads
                pair_mask = pair_mask & sep_mask
            elif sepcut == "high":
                sep_mask = seps > 0.5*pairGrRads
                pair_mask = pair_mask & sep_mask
                               
            pair_per_real = []
            major_per_real = []
            minor_per_real = []
                
            realizations = np.unique( pairReals )

            for real in realizations:
                pair_real = pairReals == real

                pair_mask_real = pair_mask & pair_real & (majors | minors)
                major_mask_real = pair_mask_real & majors
                minor_mask_real = pair_mask_real & minors

                pair_xx = np.median( keydat[pair_mask_real] )
                major_xx = np.median( keydat[major_mask_real] )
                minor_xx = np.median( keydat[minor_mask_real] )

                pair_per_real.append( pair_xx )
                major_per_real.append( major_xx )
                minor_per_real.append( minor_xx )

            med_pairs_snap = np.median(pair_per_real)
            med_major_snap = np.median(major_per_real)
            med_minor_snap = np.median(minor_per_real)
            quart_pairs_snap = np.percentile( pair_per_real,[16,84] ) 
            quart_major_snap = np.percentile( major_per_real,[16,84] ) 
            quart_minor_snap = np.percentile( minor_per_real,[16,84] ) 
                
            redshifts.append( redshift )
            med_pairs.append( med_pairs_snap )
            med_major.append( med_major_snap )
            med_minor.append( med_minor_snap )

            quart_pairs.append( quart_pairs_snap )
            quart_major.append( quart_major_snap )
            quart_minor.append( quart_minor_snap )

        except KeyError:
            if errorprint: print(f'skipping {snap} for KeyError. Please debug')
            continue
            
        except EmptyFile:
            if errorprint: print(f"skipping {snap}, empty file")
            continue
            
        except SkipRedshift:
            if errorprint: print(f"skipping {snap}, redshift out of range")
                
                
    sep_dictionary = {"z":np.array(redshifts),
                        "All Median": np.array(med_pairs),
                        "Major Median": np.array(med_major),
                        "Minor Median":np.array(med_minor),
                        "All Quarts":np.array(quart_pairs),
                        "Major Quarts":np.array(quart_major),
                        "Minor Quarts":np.array(quart_minor)}
    
    return sep_dictionary

def get_key_simstell(sim, size, key, sepcut, errorprint=False, redshiftcutoff=True):    
    if sim == "Illustris":
        snapshots = np.arange(0,136,1)
        snapshots = np.delete(snapshots, np.where(snapshots==113)[0])
        redcutoff = 3.8
    elif sim == "TNG":
        snapshots = np.arange(0,100,1)
        snapshots = np.delete(snapshots, np.where(snapshots==48)[0])
        redcutoff = 4
        
    redshifts = []  
    med_pairs, med_major, med_minor = [], [], []
    quart_pairs, quart_major, quart_minor = [], [], []
            
    for snap in snapshots:  
        try:
            pair_path = f"{sim}_{snap}_simstars.hdf5"
            pair_data = h5py.File(f"{paths.path_pairs}{pair_path}", "r")
            
            if np.size(pair_data) == 0:
                raise EmptyFile
                
            redshift = pair_data['Header'].attrs['Redshift']
            
            if redshiftcutoff & ( redshift > redcutoff) :
                raise SkipRedshift
                
            if (len(pair_data['pairs']['hydro']['Group ID']) == 0):    
                raise EmptyFile
                
            pair = pair_data["pairs"]['hydro']
            priStell = np.array(pair["Sub1 Stellar Mass"])
            secStell = np.array(pair["Sub2 Stellar Mass"])
            pairGroups = np.array(pair["Group Mass"])
            pairGrRads = np.array(pair["Group Radius"])
            seps = np.array(pair["Separation"]) 
            keydat = np.array(pair[key])
            
            # subset masks for unpaired
            pair_pri = get_primmask(priStell, size)
            pair_group = get_groupmask(pairGroups, size)

            majors = (secStell/priStell > 1/4)
            minors = (secStell/priStell > 1/10) & (secStell/priStell < 1/4)
            pair_lowsep = (seps > 10) & (seps < 10000)

            # defining combined masks 
            pair_mask = pair_pri & pair_group & pair_lowsep
        
            if sepcut == "low":
                sep_mask = seps < 0.5*pairGrRads
                pair_mask = pair_mask & sep_mask
            elif sepcut == "high":
                sep_mask = seps > 0.5*pairGrRads
                pair_mask = pair_mask & sep_mask
                               


            pair_mask = pair_mask & (majors | minors)
            major_mask = pair_mask & majors
            minor_mask = pair_mask & minors

            pair_dat = keydat[pair_mask]
            major_dat = keydat[major_mask]
            minor_dat = keydat[minor_mask]

            med_pairs_snap = np.median(pair_dat)
            med_major_snap = np.median(major_dat)
            med_minor_snap = np.median(minor_dat)
            quart_pairs_snap = np.percentile( pair_dat,[16,84] ) 
            quart_major_snap = np.percentile( major_dat,[16,84] ) 
            quart_minor_snap = np.percentile( minor_dat,[16,84] ) 
                
            redshifts.append( redshift )
            med_pairs.append( med_pairs_snap )
            med_major.append( med_major_snap )
            med_minor.append( med_minor_snap )

            quart_pairs.append( quart_pairs_snap )
            quart_major.append( quart_major_snap )
            quart_minor.append( quart_minor_snap )

        except KeyError:
            if errorprint: print(f'skipping {snap} for KeyError. Please debug')
            continue
            
        except EmptyFile:
            if errorprint: print(f"skipping {snap}, empty file")
            continue
            
        except SkipRedshift:
            if errorprint: print(f"skipping {snap}, redshift out of range")
                
                
    sep_dictionary = {"z":np.array(redshifts),
                        "All Median": np.array(med_pairs),
                        "Major Median": np.array(med_major),
                        "Minor Median":np.array(med_minor),
                        "All Quarts":np.array(quart_pairs),
                        "Major Quarts":np.array(quart_major),
                        "Minor Quarts":np.array(quart_minor)}
    
    return sep_dictionary

In [ ]:
path = f"Illustris_135_simstars.hdf5"
data = h5py.File(f"{paths.path_pairs}{path}", "r")